### Optional

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import time
import re
import os
import sys
import urllib.request
import json
import pandas as pd

### Characters

In [ ]:
def number_of_maincharacters(dramas):
    two_characters =[]
    three_characters =[]
    four_characters =[]
    for _, item in dramas.iterrows():
        if type(item[6]) != str:
            two_characters.append(item[1])
        if type(item[6]) == str and type(item[8]) != str:
            three_characters.append(item[1])
        if type(item[8]) == str:
            four_characters.append(item[1])
    return two_characters, three_characters, four_characters

def get_main_characters(dramas, two_characters, three_characters, four_characters):
    # 참조할 df 생성
    characters = dramas[['drama_name', 'character1', 'actor1', 'character2', 'actor2', 'character3', 'actor3', 'character4', 'actor4']]
    main_characters = {}
    for _, item in characters.iterrows():
        if item[0] in four_characters:
            lst = item[1:].to_list()
        if item[0] in three_characters:
            lst = item[1:7].to_list()    
        if item[0] not in two_characters:
            lst = item[1:5].to_list()
        main_characters[item[0]] = lst
    return main_characters

### episode_numbers

In [ ]:
def drama_names(dramas):
    drama_name_list = dramas.drama_name.to_list()
    episode_num_list = dramas.total_episode_num.to_list()
    # len(drama_name_list) == len(episode_num_list)
    return drama_name_list, episode_num_list

def get_total_episode_num(drama_name_list, episode_num_list):
    drama_tot_ep_num = {}
    for i in range(len(drama_name_list)):
        drama_tot_ep_num[drama_name_list[i]] = int(episode_num_list[i])
    return drama_tot_ep_num

### Naver API

In [ ]:
# 신청하신 API 정보 기입
Naver_client_id = ""
Naver_client_secret = ""

### Preprocessing functions

In [ ]:
def delete_iframe(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 
    src_url = "https://blog.naver.com/" + soup.iframe["src"]
    return src_url

def get_stories(review_list, start_words, end_words):
    """
    review_list : 각 태그 문장들을 분리해놓은 리스트
    start_words : 줄거리 요약의 시작을 알리는 단어들의 리스트
    end_words : 줄거리 요약의 끝을 알리는 단어들의 리스트    
    """
    start_idx = 0
    end_idx = len(review_list) -1
    for item in review_list:
        for startword in start_words:
            p = re.compile(startword)
            # p.match("string") 의 일치 결과 없으면 None을 return
            if p.match(item):
                start_idx = max(review_list.index(item), start_idx)
        for endword in end_words:
            q = re.compile(endword)
            if q.match(item):
                end_idx = min(review_list.index(item), end_idx)

    if start_idx < end_idx:
        stories = " ".join(review_list[start_idx:end_idx+1]) # 원래 형태로 복원
    else:
        # 뒤쪽에 광고가 나오는 경우 많으므로 애매하면 뒤쪽만 없애기
        stories = " ".join(review_list[:end_idx+1])
    
    return stories

def is_not_sentence(df):
    '''
    문장의 후보로 판단된 대상들 중 2어절 이하인 경우 완전한 문장이 아니라 판단해 drop
    '''
    cnt = 0
    notasentence = []
    for idx, row in df.iterrows():
        item = row['sentence']
        item = re.sub(' +', ' ', item) # 공백 정규화

        # 해쉬태크 등 어절단위가 2개 보다 적은경우 해당 row의 인덱스를 저장. (해쉬태그거나 감정이 드러나지 않을 가능성 높음.)
        if len(item.strip().split(' ')) < 3 :
            notasentence.append(idx)
            cnt += 1
    return cnt, notasentence
 
def get_only_sentences(df, notasentence):
    '''
    문장이 아닌 인덱스의 리스트를 바탕으로 해당 행을 삭제
    '''
    lst = df['sentence'].to_list()
    # null_idx에 해당하는 행 삭제
    for i in tqdm(range(len(notasentence))):
        df.drop(notasentence[i], inplace=True)
    a, _ = is_not_sentence(df)
    if a == 0:
        return df

### naverblog_drama_episode_scraper

In [ ]:
def naverblog_drama_episode_scraper(drama_name_list, main_characters, year, skip_words, max_blog_num=5):
    '''
    drama_name_list : 검색할 드라마에 대한 이름
    main_characters : 드라마 별 주연 배우의 배역명, 본명을 담는 딕셔너리
    year : 검색 결과를 저장할 파일에 지정할 연도 이름
    skip_words : 기타 불필요하다고 생각했던 어투가 문장에 있는 경우 제거
    '''
    client_id = Naver_client_id
    client_secret = Naver_client_secret
    #틀 만들기
    df = pd.DataFrame(columns= ['drama_name','episode_num','blog_url', 'sentence']) # , 'count_main_characters'
    # drama_ep_reviews = {}
    # 드라마 이름 선택
    for k in tqdm(range(len(drama_name_list))):
        drama_name = drama_name_list[k]
        actor_names = main_characters[drama_name] # main_actor list

        # 회차 수 지정
        for i in tqdm(range(1, drama_tot_ep_num[drama_name]+1), mininterval=0.1):
            encText = urllib.parse.quote(drama_name+f' "{i}회 줄거리"')
            url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&display=25" # json 결과, display default 10
            # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            if(rescode==200):
                response_body = response.read()
                text_data = response_body.decode('utf-8')
                # print(crawling_base)
            else:
                print("Error Code:" + rescode)
            json_data = json.loads(text_data)
            # display(json_data)

            max_j = len(json_data['items'])
            # 관련 블로그 검색이 너무 적은 경우에는 해당 드라마는 검색하지 않음 : 추후 회차 부족한 드라마 필터링 시에 같이 삭제
            if max_j < 5:
                break
            cnt = 0
            blog_urls = []
            for j in tqdm(range(max_j)):
                # 'https://blog.naver.com/shyhjin?Redirect=Log&logNo=222214746826' : X
                if "?Redirect=Log&logNo=" in json_data['items'][j]['link']:
                    blog_url = json_data['items'][j]['link'].replace("?Redirect=Log&logNo=","/")
                else:
                    blog_url = json_data['items'][j]['link']
                if "tistory.com" in blog_url:
                    continue
                blog_urls.append(blog_url)
                cnt += 1
                if cnt == max_blog_num: # Edit, 불러올 블로그 개수
                    break
            # blog_urls

            # url 통해 blog 내의 글에 접근하기
            for target_url in blog_urls:
                time.sleep(0.5)
                # naver blog는 iframe으로 담겨있는 경우가 있어 이를 해제해주어야 할 수 있음
                try:
                    tmp_res = requests.get(delete_iframe(target_url))
                except:
                    print("delete_iframe not used!")
                    tmp_res = requests.get(target_url)
                # status check
                if tmp_res.status_code != 200:
                    continue
                # bs4
                tmp_soup = BeautifulSoup(tmp_res.text, 'html.parser')
                # se-component se-text
                if tmp_soup.find('div', class_='se-main-container') == None:
                    continue
                sentence_tags = tmp_soup.find('div', class_='se-main-container').find_all('p', class_='se-text-paragraph') # 각 blog의 sentences담은 tag -> list
                
                raw_corpus = [] # list containing texts in each tag
                for sentence_tag in sentence_tags:
                    # 해당 태그가 없는 경우 skip
                    if sentence_tag == None:
                        continue
                    # 해당 태그 내의 text가 공란이 아닌 경우만 반영
                    res = sentence_tag.text
                    # res.replace('\u200b', '')
                    if res.replace(' ', '') != '\u200b' and res.replace(' ', '') != '':
                        raw_corpus.append(res)
                
                # f'{i-1}회', f'{i-1}화', 
                start_words = ['줄거리 요약 시작','강스포(0)','스포를 원치 않으신다면 여기서 뒤로가기를 눌러주세요 !','줄거리 & 리뷰','리뷰 시작', '줄거리를 보고 싶은 분만 내리세요.']
                end_words = ['@레몬스카이','감사합니다 !', '포스팅 끝', '예고 공식영상', f'{i+1}회예고',
                            '사진 출처 │ tvN 드라마 공식홈페이지 및 티빙 캡처 *본 리뷰에 인용된 대사 및 사진의 저작권은 방송사 tvN에 있습니다.',
                            '{i+1}화 에필로그','{i+1}회 예고편', '아래 포스팅 클릭', '이미지, 영상 출처/저작권', '오늘도 읽어주셔서 감사합니다 !',
                            '후기 리뷰에 인용된', '여기서 이만', '다음회 리뷰에서 만나요']
                
                story_corpus = get_stories(raw_corpus, start_words, end_words)
                
                # story_corpus = clean_text(story_corpus)
                story_corpus = re.sub(' +', ' ', story_corpus) # multiple space to space
                story_corpus = re.sub("https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", '', story_corpus) # url 제거
                story_corpus = re.sub(drama_name, '', story_corpus) # drama_name 제거
                story_corpus = re.sub('[(\d+)\회\차|(\d+)\회]', '', story_corpus) # 회차 정보 제거
                
                sentences_ingroup = re.split("\.|\?|\!|\~|\#|\;", story_corpus) # multiple condition split by regular expressions
                # print(sentences_ingroup) # ['길고 긴 세월 인간이 되기 위해 신우여 기생집 전전하며 여성의 정기 모으고 있었다', ' 잘생긴 외모 시크한 모습 많은 여성들 그에게 호감 표시했어요 진짜 성균관 학생', ...]

                temp_result = []
                for sentence_string in sentences_ingroup:
                    indic = 0
                    for skip_word in skip_words:
                        if sentence_string.find(skip_word) != -1:
                            indic = 1
                    if indic == 1:
                        continue
                    
                    sentence_string = re.sub(' +', ' ', sentence_string)
                    # sentence_string = re.split("' '", sentence_string.strip())
                    if sentence_string in ['', ""]:
                        continue
                    # res = is_actor_in_sentence(sentence_ingroup, actor_names)
                    # print(sentence_string)
                    # print("하나 끝")

                    temp_result.append([drama_name, i, target_url, sentence_string]) # , res
                
                df_tmp = pd.DataFrame(temp_result, columns= ['drama_name','episode_num','blog_url', 'sentence']) # , 'count_main_characters'
                df = pd.concat([df, df_tmp])
                # display(df)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

### Scraping every episode of each drama for multiple years

In [ ]:
# Initial Input Table : explaining drama's total episode number and the information of actors
# year = 2020 # Edit
for year in tqdm(range(2019, 2022)):
    dramas = pd.read_excel(f"/content/drive/MyDrive/YBIGTA/Project_Spoiler/baseline_data/dramas_{year}.xlsx")

    two_characters, three_characters, four_characters = number_of_maincharacters(dramas)
    main_characters = get_main_characters(dramas, two_characters, three_characters, four_characters)
    drama_name_list, episode_num_list = drama_names(dramas)
    drama_tot_ep_num = get_total_episode_num(drama_name_list, episode_num_list)
    # skip when words below appear # string method .find()
    skip_words = ['강스포(0)', ' (문제시 삭제하도록 하겠습니다)', ' 저작권은', ' 리뷰 후기',
                '리뷰로 또 찾아올게요', '레몬스카이', '팬해주시면 행복', '(스포0)',
                '포스팅 더 보기', '리뷰에서 만나요', '리뷰 마치도록 하겠습니다', '예고공식영상',
                '프로필', '사진출처는', '넷플릭스', '펜트하우스', '공식홈페이지', '드라마', '예고',
                '출연', '스와로브스키', '드미어 핸드메이드', '우영우']

    df = naverblog_drama_episode_scraper(drama_name_list, main_characters, year, skip_words, 5)
    _, notasentence = is_not_sentence(df)
    df = get_only_sentences(df, notasentence)

    df['sentence'] = [re.sub('[^0-9가-힣ㄱ-ㅎ]', ' ', s) for s in df['sentence']]#한글과 숫자 제외하고 없애기
    df['sentence'] = [re.sub('ㅋㅋ+', 'ㅋㅋ', s) for s in df['sentence']] # ㅋ을 남발한 경우
    df['sentence'] = [re.sub('ㅎㅎ+', 'ㅎㅎ', s) for s in df['sentence']] # ㅎ을 남발한 경우
    df['sentence'] = [re.sub(' +', ' ', s) for s in df['sentence']]  # 빈칸 크기 정규화

    _, notasentence = is_not_sentence(df)
    df = get_only_sentences(df, notasentence)

    display(df)
    df.to_csv(f"/content/drive/MyDrive/YBIGTA/Project_Spoiler/Crawling/crawled_data/{year}_dramareviews_by_context(another_version).csv", index=False)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]


 20%|██        | 5/25 [00:00<00:00, 34267.19it/s]


  6%|▋         | 1/16 [00:07<01:45,  7.02s/it]


 20%|██        | 5/25 [00:00<00:00, 32615.12it/s]


 12%|█▎        | 2/16 [00:14<01:44,  7.50s/it]


 24%|██▍       | 6/25 [00:00<00:00, 35345.26it/s]


 19%|█▉        | 3/16 [00:21<01:30,  6.98s/it]


 20%|██        | 5/25 [00:00<00:00, 30570.73it/s]


 25%|██▌       | 4/16 [00:27<01:20,  6.73s/it]


 24%|██▍       | 6/25 [00:00<00:00, 39138.14it/s]


 31%|███▏      | 5/16 [00:34<01:15,  6.85s/it]


 28%|██▊       | 7/25 [00:00<00:00, 49097.20it/s]


 38%|███▊      | 6/16 [00:41<01:10,  7.01s/it]


 24%|██▍       | 6/25 [00:00<00:00, 43539.49it/s]


 44%|████▍     | 7/16 [00:50<01:06,  7.36s/it]


 20%|██        | 5/25 [00:00<00:00, 34323.27it/s]


 50%|█████     | 8/16 [00:58<01:02,  7.82s/it]


 20%|██        | 5/25 [00:00<00:00, 28649.62it/s]


 56%|█████▋    | 9/16 [01:08<00:57,  8.26s/it]


 24%|██▍  

,drama_name,episode_num,blog_url,sentence
0,사랑의 불시착,1,https://blog.naver.com/tjdud0013/222551136125,윤세리 대한민국 굴지의 재벌가 남 녀중 막내딸로 나오게 되지만 알고보면 아빠 윤증...
1,사랑의 불시착,1,https://blog.naver.com/tjdud0013/222551136125,그리고 집행유예 받고 난 이후 윤증평은 후계자 자리를 계획하게 되는데요 그렇게 세...
2,사랑의 불시착,1,https://blog.naver.com/tjdud0013/222551136125,때마침 등장하게 된 세리 아버지 집행유예 받으신거 축하드려요 저 불으셨다고 들어서...
3,사랑의 불시착,1,https://blog.naver.com/tjdud0013/222551136125,얼굴 좋아보이시네요 건강잘 챙기시구요 인사드렸으니까 전 가볼게요 하지만 그 순간 ...
4,사랑의 불시착,1,https://blog.naver.com/tjdud0013/222551136125,잘됐네요 몇몇 대표들은 자격이 없다 생각했거든요 직원들의 불화나 본인의 무지로 인...
...,...,...,...,...
95998,그녀의 사생활,16,https://blog.naver.com/cwmylee/222847811229,내가 이 짓을 해서라도 너를 꼭 보러 가겠다고 합니다 거기에 대한 답장이야
95999,그녀의 사생활,16,https://blog.naver.com/cwmylee/222847811229,나도 보고 싶었다 도련님도 제자도 아닌 한 남자로서 무덕이를 향한 연심을 고백했지...
96001,그녀의 사생활,16,https://blog.naver.com/cwmylee/222847811229,지금 폭주한 거면 나를 만지면 너는 물기를 빼앗겨 죽을 꺼라고 했고 장욱은 그런 ...
96002,그녀의 사생활,16,https://blog.naver.com/cwmylee/222847811229,이어 단근 수련을 하지 않는다며 자신을 끌어안는 장욱에게 무덕이는 내가 어찌 너의...


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


 20%|██        | 5/25 [00:00<00:00, 26051.58it/s]


 31%|███▏      | 5/16 [00:37<01:24,  7.64s/it]


 24%|██▍       | 6/25 [00:00<00:00, 36314.32it/s]


 38%|███▊      | 6/16 [00:45<01:16,  7.64s/it]


 20%|██        | 5/25 [00:00<00:00, 32564.47it/s]


 44%|████▍     | 7/16 [00:52<01:08,  7.62s/it]


 20%|██        | 5/25 [00:00<00:00, 29916.58it/s]


 50%|█████     | 8/16 [01:00<01:01,  7.66s/it]


 20%|██        | 5/25 [00:00<00:00, 33662.15it/s]


 56%|█████▋    | 9/16 [01:07<00:52,  7.52s/it]


 20%|██        | 5/25 [00:00<00:00, 32922.32it/s]


 62%|██████▎   | 10/16 [01:17<00:48,  8.11s/it]


 20%|██        | 5/25 [00:00<00:00, 32716.88it/s]


 69%|██████▉   | 11/16 [01:26<00:41,  8.32s/it]


 20%|██        | 5/25 [00:00<00:00, 31254.13it/s]


 75%|███████▌  | 12/16 [01:34<00:32,  8.23s/it]


 20%|██        | 5/25 [00:00<00:00, 32819.28it/s]


 81%|████████▏ | 13/16 [01:43<00:25,  8.47s/it]


 20%|██        | 5/25 [00:00<00:00, 32363.46it/s]

,drama_name,episode_num,blog_url,sentence
6,런 온,1,https://blog.naver.com/shj0504/222176062966,오미주 기선겸 첫 만남 통번역을 전문으로 하는 오미주신세경는 자신이 번역한 영화를...
7,런 온,1,https://blog.naver.com/shj0504/222176062966,뒤풀이에 참가하다 교수님과 한바탕 난리가 나고 혼자 돌아가던 중 기선겸임시완과부딪힌다
8,런 온,1,https://blog.naver.com/shj0504/222176062966,쫓아오던 구남친이 미주의 팔목을 잡자 미주의 총 모양 라이터를 그에게 들이대는데
9,런 온,1,https://blog.naver.com/shj0504/222176062966,두 번째 만남 직거래를 하던 중 상대방의 사기 행각에 열심히 뒤쫓아 달리는 오미주
10,런 온,1,https://blog.naver.com/shj0504/222176062966,미주를 본 선겸은 사기꾼을 한 번에 잡는다
...,...,...,...,...
135852,머니게임,16,https://blog.naver.com/bortori90/221839378615,돌파 방안을 위해 허재를 찾은 이혜준에게 허재는 바하마를 무너뜨릴 수 있는 마지막...
135853,머니게임,16,https://blog.naver.com/bortori90/221839378615,니가 얼마나 불행한지 봐
135854,머니게임,16,https://blog.naver.com/bortori90/221839378615,죽이고 싶은 유진한유태오 향한 채이헌고수의 마지막 경고
135855,머니게임,16,https://blog.naver.com/bortori90/221839378615,결정적 한 방


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


 67%|██████▋   | 4/6 [00:00<00:00, 25003.30it/s]


 56%|█████▋    | 9/16 [00:59<00:47,  6.74s/it]


 80%|████████  | 4/5 [00:00<00:00, 29852.70it/s]


 62%|██████▎   | 10/16 [01:06<00:41,  6.85s/it]


 67%|██████▋   | 4/6 [00:00<00:00, 37532.92it/s]


 69%|██████▉   | 11/16 [01:13<00:34,  6.92s/it]


 80%|████████  | 4/5 [00:00<00:00, 34169.48it/s]


 75%|███████▌  | 12/16 [01:20<00:27,  6.93s/it]


 67%|██████▋   | 4/6 [00:00<00:00, 30840.47it/s]


 81%|████████▏ | 13/16 [01:27<00:20,  6.88s/it]


 67%|██████▋   | 4/6 [00:00<00:00, 28630.06it/s]


 88%|████████▊ | 14/16 [01:33<00:13,  6.83s/it]


 67%|██████▋   | 4/6 [00:00<00:00, 29694.19it/s]


 94%|█████████▍| 15/16 [01:40<00:06,  6.73s/it]

 57%|█████▋    | 17/30 [31:55<20:25, 94.26s/it]

  0%|          | 0/16 [00:00<?, ?it/s]


 16%|█▌        | 4/25 [00:00<00:00, 28926.23it/s]


  6%|▋         | 1/16 [00:07<01:55,  7.73s/it]


 16%|█▌        | 4/25 [00:00<00:00, 30393.51it/s]


 12%|█▎       

,drama_name,episode_num,blog_url,sentence
0,간 떨어지는 동거,1,https://blog.naver.com/werjazz/222369864268,길고 긴 세월 인간이 되기 위해 신우여 기생집 전전하며 여성의 정기 모으고 있었다
1,간 떨어지는 동거,1,https://blog.naver.com/werjazz/222369864268,잘생긴 외모 시크한 모습 많은 여성들 그에게 호감 표시했어요 진짜 성균관 학생
2,간 떨어지는 동거,1,https://blog.naver.com/werjazz/222369864268,극중 신우여 장기용 덕을 쌓아 년 지나면 꼬리 하나 생기고 꼬리 아홉 개 넘기기 ...
3,간 떨어지는 동거,1,https://blog.naver.com/werjazz/222369864268,극중 양예선 년 만에 인간 된 것 보면 꼭 년 기다리지 않았도 되나 봐요
4,간 떨어지는 동거,1,https://blog.naver.com/werjazz/222369864268,신우여 직업 해밀 출판사 작가 크리 에디터 역사 관련 일을 하고 있으며 밥을 먹지...
...,...,...,...,...
126436,크레이지 러브,14,https://blog.naver.com/doden55ro/222705673126,노고진의 앞에 나타난 한 여성
126437,크레이지 러브,14,https://blog.naver.com/doden55ro/222705673126,신아가 이 분 보고 나수연
126439,크레이지 러브,14,https://blog.naver.com/doden55ro/222705673126,이라고 불렀는데 선생님이면
126440,크레이지 러브,14,https://blog.naver.com/doden55ro/222705673126,고탑에 있던 강사님일까요


100%|██████████| 3/3 [2:38:21<00:00, 3167.02s/it]
